## Dependencies

In [8]:
import wikipedia
from tqdm import tqdm
import numpy as np

## Data loading

Create corresponding labels.

In [16]:
# 1 if Western Armenian, 0 otherwise
# y_western = np.ones(NUM_PAGES_WESTERN)
# y_eastern = np.zeros(NUM_PAGES_EASTERN)

# y = np.concatenate((y_western, y_eastern))